## Step 1 

Get the library and set the seed

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# reset the graph and make sure the random numbers are always the same

def reset_graph(seed=2018):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Step 2 

- Reset graph
- Hyperparameters
- Network Parameters
- Build RNN Model (place holders, cell, logits, cross entropy, optimizer, accuracy

In [3]:
# reset
reset_graph()

In [4]:
# hyperparameters
n_neurons = 256
learning_rate = 0.001
batch_size = 128
n_epochs = 100

# parameters
n_steps = 28 # 28 rows
n_inputs = 28 # 28 cols
n_output = 10 # 10 classes

In [9]:
# build a rnn model
# Placeholders
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

# Cell - use basic RNN cell
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# logits, cross entropy, loss, optimizer
logits = tf.layers.dense(state, n_output)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels =y,
                                                              logits=logits)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
# prediction, accuracy

prediction = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [6]:
# input data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

X_test = mnist.test.images
X_test = X_test.reshape([-1, n_steps, n_inputs])
y_test = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Step 3: Initialize and Run our model 

In [ ]:
# initialize the variables
init = tf.global_variables_initializer()

#train
sess = tf.Session()
sess.run(init)
n_batches = mnist.train.num_examples // batch_size
loss_list = []
acc_list = []

for epoch in range(n_epochs):
    for batch in range(n_batches):
        X_train, y_train = mnist.train.next_batch(batch_size)
        X_train          = X_train.reshape([-1, n_steps, n_inputs])
        sess.run(optimizer, feed_dict={X:X_train, y:y_train})
    loss_train, acc_train = sess.run( [loss,accuracy], feed_dict={X:X_train, y:y_train})
    loss_list.append(loss_train)
    acc_list.append(acc_train)
    print('Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch +1, 
                                                                    loss_train,
                                                                    acc_train))
    
loss_test, acc_test = sess.run([loss, accuracy], feed_dict={X:X_test, y:y_test})
print('Test loss: {:.4f}, Test acc: {:.4f}'.format(loss_test, acc_test))

Epoch: 1, Train Loss: 0.1636, Train Acc: 0.9688
Epoch: 2, Train Loss: 0.0748, Train Acc: 0.9766
Epoch: 3, Train Loss: 0.0718, Train Acc: 0.9766
Epoch: 4, Train Loss: 0.0926, Train Acc: 0.9688
Epoch: 5, Train Loss: 0.0492, Train Acc: 0.9922
Epoch: 6, Train Loss: 0.0724, Train Acc: 0.9844
Epoch: 7, Train Loss: 0.0499, Train Acc: 0.9844
Epoch: 8, Train Loss: 0.0586, Train Acc: 0.9844
Epoch: 9, Train Loss: 0.0282, Train Acc: 0.9922
Epoch: 10, Train Loss: 0.0757, Train Acc: 0.9766
Epoch: 11, Train Loss: 0.0113, Train Acc: 1.0000
Epoch: 12, Train Loss: 0.0305, Train Acc: 0.9922
Epoch: 13, Train Loss: 0.0180, Train Acc: 0.9922
Epoch: 14, Train Loss: 0.0404, Train Acc: 0.9922
Epoch: 15, Train Loss: 0.0403, Train Acc: 0.9766
Epoch: 16, Train Loss: 0.0209, Train Acc: 0.9922
Epoch: 17, Train Loss: 0.0983, Train Acc: 0.9844
Epoch: 18, Train Loss: 0.0212, Train Acc: 0.9922
Epoch: 19, Train Loss: 0.0505, Train Acc: 0.9922
Epoch: 20, Train Loss: 0.0632, Train Acc: 0.9844
Epoch: 21, Train Loss: 0.0137

## Step 4: PLot Accuracy 

In [ ]:
# plot train loss vs epoch
plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.title('Train Loss vs Epoch', fontsize=15)
plt.plot(np.arange(n_epochs), loss_list, 'r-')
plt.xlabel('Epoch')
plt.ylabel('Train Loss')

# plot train accuracy vs epoch
plt.subplot(1, 2, 2)
plt.title('Train Accuracy vs Epoch', fontsize=15)
plt.plot(np.arange(n_epochs), acc_list, 'b-')
plt.xlabel('Epoch')
plt.ylabel('Train Accuracy')
plt.show()

## Step 5: Let's output the RNN predictions 

In [ ]:
n_predict = 10 # number to display prediction

for i in range(n_predict):
    pred = sess.run(tf.argmax(logits[i]), feed_dict={X: X_test, y: y_test})
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.show()
    print('predict: ', pred)

In [ ]:
sess.close()